In [ ]:
import pandas as pd
import sqlalchemy as sa
import numpy as np
import json

In [ ]:
%run ../toxref_mysql_connect.py
# print(connect.__doc__)
con = connect()  

In [ ]:
query = """
    SELECT 
        study.study_id,
        study.study_type,
        tg.tg_id,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.generation,
        study.species,
        dose.conc,
        dose.conc_unit,
        tg.sex,
        chemical.dsstox_substance_id,
        chemical.casrn,
        chemical.preferred_name,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_desc,
        tg_effect.effect_desc_free,
        tg_effect.life_stage,
        tg_effect.target_site,
        tg_effect.direction,
        dose.dose_level,
        endpoint.endpoint_id,
        effect.effect_id,
        tg_effect.tg_effect_id,
        dtg.dose_adjusted,
        dtg.dose_adjusted_unit,
        dtg.mg_kg_day_value,
        dtg_effect.treatment_related,
        dtg_effect.critical_effect
    FROM 
        study INNER JOIN dose ON dose.study_id=study.study_id
			INNER JOIN chemical ON chemical.chemical_id=study.chemical_id
				INNER JOIN tg ON tg.study_id=study.study_id
					INNER JOIN dtg ON tg.tg_id=dtg.tg_id AND dose.dose_id=dtg.dose_id
						LEFT JOIN tg_effect ON tg.tg_id=tg_effect.tg_id
							LEFT JOIN dtg_effect ON tg_effect.tg_effect_id=dtg_effect.tg_effect_id AND dtg.dtg_id=dtg_effect.dtg_id
								LEFT JOIN effect ON effect.effect_id=tg_effect.effect_id
									LEFT JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id
    AND study.study_id NOT IN(6940,6955) #pfizer studies with incomplete data
    ;
                
        """

In [ ]:
#configs
effect_profile_id = 2
pod_level = "chemical" #study vs chemical level pods

In [ ]:
studies = pd.read_sql("study",con)
effect_profile = pd.read_sql("effect_profile",con).merge(
    pd.read_sql("effect_profile_group",con)).merge(
        pd.read_sql("effect_profile_group_toxrefdb",con))
effect_profile = effect_profile[effect_profile.effect_profile_id==effect_profile_id].copy()
df = pd.read_sql(query,con)

In [ ]:
# exclude studies that fail the nel<=lel and/or noael<=loael checks due to stagered dose adjusted based on gender
# df = df[df["study_id"].isin([1450,552,2764,2766,3879,3126,4450,542,2764,4632,4856])==False].copy()

df = df.merge(effect_profile,how="left")

# if effect_profile_id==1:
#     df = df.merge(effect_profile,how="left",left_on="tg_effect_id",right_on="toxrefdb_id")
# elif effect_profile_id==2:
#     df = df.merge(effect_profile,left_on="endpoint_id",right_on="toxrefdb_id")

In [ ]:
if pod_level=="study":
    df_index_names = ["dsstox_substance_id","study_id","group_id"]
elif pod_level=="chemical":
    df_index_names = ["dsstox_substance_id","group_id"]
df_column_names = ["pod_type","qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level","max_dose_level","tg_effect_ids"]

In [ ]:
# update the missing dose_adjusted and corresponding unit with the conc and conc_unit
df.update(pd.Series(df[df.dose_adjusted.isnull()].conc_unit.values, 
                    name="dose_adjusted_unit",
                    index=df[df.dose_adjusted.isnull()].index.values))
df.update(pd.Series(df[df.dose_adjusted.isnull()].conc.values, 
                    name="dose_adjusted",
                    index=df[df.dose_adjusted.isnull()].index.values))
df.fillna("nan",inplace=True)

In [ ]:
def loael(x):
    t = "loael"
    x = x.replace("nan",np.nan)
    if len(x[x.critical_effect==1]) > 0:
        qual = "'='"
        val = x[x.critical_effect==1].dose_adjusted.min()
        indx = x[(x.critical_effect==1)&(x.dose_adjusted==val)].index.values[0]
        indxs = x[(x.critical_effect==1)&(x.dose_adjusted==val)].index.values
        tg_id = x.loc[indx].tg_id
        val_dl = x.loc[indx].dose_level

        dls = x[x.tg_id==tg_id].dose_level.sort_values().unique()
        dl_indx = list(dls).index(val_dl)

        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        max_dl = dls[-1]
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x[x.critical_effect==1].dose_adjusted.min()][["toxrefdb_id",
#                                                                                            "toxrefdb_table"]].values
    else: #check to make sure it's not an empty dataframe
        qual = ">"
        val_dl = x.dose_level.max()
        val = x[x.dose_level==val_dl].dose_adjusted.min()
        if not np.isnan(val):
            indx = x[x.dose_adjusted==val].index.values[0]
            indxs = x[x.dose_adjusted==val].index.values
        else:
            indx = x[x.dose_level==val_dl].index.values[0]
            indxs = x[x.dose_level==val_dl].index.values
        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        val_dl = x.dose_level.max()
        max_dl = val_dl
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x.dose_adjusted.max()][["toxrefdb_id","toxrefdb_table"]].values
#     return (t,qual,val,val_unit,val_dl,max_dl,tg_effect_ids,toxrefdb_refs)
    return (t,qual,val,val_unit,mkd,val_dl,max_dl,tg_effect_ids)

In [ ]:
def noael(x):
    t = "noael"
    x = x.replace("nan",np.nan)
    if len(x[x.critical_effect==1]) > 0:
        val = x[x.critical_effect==1].dose_adjusted.min()
        indx = x[(x.critical_effect==1)&(x.dose_adjusted==val)].index.values[0]
        indxs = x[(x.critical_effect==1)&(x.dose_adjusted==val)].index.values
        tg_id = x.loc[indx].tg_id
        val_dl = x.loc[indx].dose_level
        
        dls = x[x.tg_id==tg_id].dose_level.sort_values().unique()
        dl_indx = list(dls).index(val_dl)
        max_dl = dls[-1]
        
        if dls[dl_indx-1] == 0: # means ce is at lowest dose (dose_level 1)
            qual = "<"
            val_unit = x.loc[indx].dose_adjusted_unit
            mkd = x.loc[indx].mg_kg_day_value

#             toxrefdb_refs = x[x.dose_adjusted==x[x.dose_level==dl+1].dose_adjusted.min()][["toxrefdb_id",
#                                                                                          "toxrefdb_table"]].values
            tg_effect_ids = x.loc[indxs].tg_effect_id.values
        else:
            qual = "'='"
            val_dl = dls[dl_indx-1]
            indx = x[(x.tg_id==tg_id)&(x.dose_level==val_dl)].index.values[0]
            indxs = x[(x.tg_id==tg_id)&(x.dose_level==val_dl)].index.values
            val = x.loc[indx].dose_adjusted
            val_unit = x.loc[indx].dose_adjusted_unit
            mkd = x.loc[indx].mg_kg_day_value
            
#             toxrefdb_refs = x[x.dose_adjusted==x[x.dose_level==dl].dose_adjusted.max()][["toxrefdb_id",
#                                                                                          "toxrefdb_table"]].values
            tg_effect_ids = x.loc[indxs].tg_effect_id.values
    else:
        qual = ">="
        val_dl = x.dose_level.max()
        val = x[x.dose_level==val_dl].dose_adjusted.min()
        if not np.isnan(val):
            indx = x[x.dose_adjusted==val].index.values[0]
            indxs = x[x.dose_adjusted==val].index.values
        else:
            indx = x[x.dose_level==val_dl].index.values[0]
            indxs = x[x.dose_level==val_dl].index.values
        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        val_dl = x.dose_level.max()
        max_dl = val_dl
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x.dose_adjusted.max()][["toxrefdb_id","toxrefdb_table"]].values
#     return (t,qual,val,val_unit,val_dl,max_dl,toxrefdb_refs)
    return (t,qual,val,val_unit,mkd,val_dl,max_dl,tg_effect_ids)

In [ ]:
def lel(x):
    t = "lel"
    x = x.replace("nan",np.nan)
    if len(x[x.treatment_related==1]) > 0:
        qual = "'='"
        val = x[x.treatment_related==1].dose_adjusted.min()
        indx = x[(x.treatment_related==1)&(x.dose_adjusted==val)].index.values[0]
        indxs = x[(x.treatment_related==1)&(x.dose_adjusted==val)].index.values
        tg_id = x.loc[indx].tg_id
        val_dl = x.loc[indx].dose_level
        
        dls = x[x.tg_id==tg_id].dose_level.sort_values().unique()
        dl_indx = list(dls).index(val_dl)
        
        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        max_dl = dls[-1]
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x[x.treatment_related==1].dose_adjusted.min()][["toxrefdb_id",
#                                                                                            "toxrefdb_table"]].values
    else:
        qual = ">"
        val_dl = x.dose_level.max()
        val = x[x.dose_level==val_dl].dose_adjusted.min()
        if not np.isnan(val):
            indx = x[x.dose_adjusted==val].index.values[0]
            indxs = x[x.dose_adjusted==val].index.values
        else:
            indx = x[x.dose_level==val_dl].index.values[0]
            indxs = x[x.dose_level==val_dl].index.values
        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        val_dl = x.dose_level.max()
        max_dl = val_dl
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x.dose_adjusted.max()][["toxrefdb_id","toxrefdb_table"]].values
#     return (t,qual,val,val_unit,val_dl,max_dl,toxrefdb_refs)
    return (t,qual,val,val_unit,mkd,val_dl,max_dl,tg_effect_ids)

In [ ]:
def nel(x):
    t = "nel"
    x = x.replace("nan",np.nan)
    if len(x[x.treatment_related==1]) > 0:
        val = x[x.treatment_related==1].dose_adjusted.min()
        indx = x[(x.treatment_related==1)&(x.dose_adjusted==val)].index.values[0]
        indxs = x[(x.treatment_related==1)&(x.dose_adjusted==val)].index.values
        tg_id = x.loc[indx].tg_id
        val_dl = x.loc[indx].dose_level
        
        dls = x[x.tg_id==tg_id].dose_level.sort_values().unique()
        dl_indx = list(dls).index(val_dl)
        max_dl = dls[-1]
        
        if dls[dl_indx-1] == 0: # means ce is at lowest dose (dose_level 1)
            qual = "<"
            val_unit = x.loc[indx].dose_adjusted_unit
            mkd = x.loc[indx].mg_kg_day_value

#             toxrefdb_refs = x[x.dose_adjusted==x[x.dose_level==dl+1].dose_adjusted.min()][["toxrefdb_id",
#                                                                                          "toxrefdb_table"]].values
            tg_effect_ids = x.loc[indxs].tg_effect_id.values
        else:
            qual = "'='"
            val_dl = dls[dl_indx-1]
            indx = x[(x.tg_id==tg_id)&(x.dose_level==val_dl)].index.values[0]
            indxs = x[(x.tg_id==tg_id)&(x.dose_level==val_dl)].index.values
            val = x.loc[indx].dose_adjusted
            val_unit = x.loc[indx].dose_adjusted_unit
            mkd = x.loc[indx].mg_kg_day_value
            
#             toxrefdb_refs = x[x.dose_adjusted==x[x.dose_level==dl].dose_adjusted.max()][["toxrefdb_id",
#                                                                                          "toxrefdb_table"]].values
            tg_effect_ids = x.loc[indxs].tg_effect_id.values
    else:
        qual = ">="
        val_dl = x.dose_level.max()
        val = x[x.dose_level==val_dl].dose_adjusted.min()
        if not np.isnan(val):
            indx = x[x.dose_adjusted==val].index.values[0]
            indxs = x[x.dose_adjusted==val].index.values
        else:
            indx = x[x.dose_level==val_dl].index.values[0]
            indxs = x[x.dose_level==val_dl].index.values
        val_unit = x.loc[indx].dose_adjusted_unit
        mkd = x.loc[indx].mg_kg_day_value
        val_dl = x.dose_level.max()
        max_dl = val_dl
        tg_effect_ids = x.loc[indxs].tg_effect_id.values
#         toxrefdb_refs = x[x.dose_adjusted==x.dose_adjusted.max()][["toxrefdb_id","toxrefdb_table"]].values
#     return (t,qual,val,val_unit,val_dl,max_dl,toxrefdb_refs)
    return (t,qual,val,val_unit,mkd,val_dl,max_dl,tg_effect_ids)

In [ ]:
df_loael = pd.DataFrame(df.groupby(df_index_names).apply(lambda x: loael(x)).to_dict()).T
df_loael.columns = df_column_names
df_loael.index.set_names(df_index_names,inplace=True)

In [ ]:
df_noael = pd.DataFrame(df.groupby(df_index_names).apply(lambda x: noael(x)).to_dict()).T
df_noael.columns = df_column_names
df_noael.index.set_names(df_index_names,inplace=True)

In [ ]:
df_lel = pd.DataFrame(df.groupby(df_index_names).apply(lambda x: lel(x)).to_dict()).T
df_lel.columns = df_column_names
df_lel.index.set_names(df_index_names,inplace=True)

In [ ]:
df_nel = pd.DataFrame(df.groupby(df_index_names).apply(lambda x: nel(x)).to_dict()).T
df_nel.columns = df_column_names
df_nel.index.set_names(df_index_names,inplace=True)

In [ ]:
pods = pd.concat([df_loael,df_noael,df_lel,df_nel])

In [ ]:
pods["dsstox_substance_id"] = pods.index.get_level_values("dsstox_substance_id")
if pod_level=="study":
    pods["study_id"] = pods.index.get_level_values("study_id")
pods["group_id"] = pods.index.get_level_values("group_id")

In [ ]:
pods = pods.merge(pd.read_sql("chemical",con),on="dsstox_substance_id")

In [ ]:
pods = pods.replace("nan",np.nan).fillna(value={"group_id":-1})
pods["effect_profile_id"] = effect_profile_id

In [ ]:
pods = pods.merge(pd.read_sql("effect_profile",con).merge(pd.read_sql("effect_profile_group",con)),how="left")

In [ ]:
# len(pods[pods.casrn.isin(missing_chems.casrn)].casrn.unique())

In [ ]:
pods["pod_value"] = pods.pod_value.astype(float)
pods["mg_kg_day_value"] = pods.mg_kg_day_value.astype(float).round(8)
pods["dose_level"] = pods.dose_level.astype(int)
pods["max_dose_level"] = pods.max_dose_level.astype(int)
pods["chemical_id"] = pods.chemical_id.astype(int)
if pod_level=="study":
    pods["study_id"] = pods.study_id.astype(int)
pods["effect_profile_id"] = pods.effect_profile_id.astype(int)
pods["group_id"] = pods.group_id.astype(int)

if pod_level=="study":
    pods[["pod_type","qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
          "max_dose_level","chemical_id","study_id",
          "effect_profile_id","group_id"]].drop_duplicates().to_sql("pod",con,if_exists="append",
                                                                                            index=False)
elif pod_level=="chemical":
    pods[["pod_type","qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
          "max_dose_level","chemical_id",
          "effect_profile_id","group_id"]].drop_duplicates().to_sql("pod",con,if_exists="append",
                                                                                            index=False)

In [ ]:
if pod_level=="study":
    pod_tbl = pd.read_sql(f"SELECT * FROM pod WHERE study_id IS NOT NULL AND effect_profile_id={effect_profile_id}",con)
elif pod_level=="chemical":
    pod_tbl = pd.read_sql(f"SELECT * FROM pod WHERE study_id IS NULL AND effect_profile_id={effect_profile_id}",con)

In [ ]:
data = []
for i,row in pod_tbl.merge(pods)[["tg_effect_ids","pod_id"]].iterrows():
    for tg_effect_id in row.tg_effect_ids:
        data.append([tg_effect_id,row.pod_id])
pd.DataFrame(data,columns=["tg_effect_id","pod_id"]).dropna().to_sql("pod_tg_effect",con,if_exists="append",index=False)